In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection
%ls

/content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection
 AdaBoostClassifier.joblib
 classifiers_recall_scores.joblib
 CNNClassifier.joblib
 DecisionTreeClassifier.joblib
 EDA.ipynb
 features_k_1000.joblib
 features_k_100.joblib
 features_k_10.joblib
 features_k_1200.joblib
 features_k_1500.joblib
 features_k_2000.joblib
 features_k_2500.joblib
 features_k_3000.joblib
 features_k_500.joblib
 features_k_50.joblib
 gnbClassifier.joblib
 GradientBoostingClassifier.joblib
 knnClassifier.joblib
 LSTMClassifier.joblib
'Macro Malware Detection using Machine Learning Techniques A New Approach '
 mlpClasifier.joblib
 __pycache__/
 randomForestClassifier.joblib
 recall_scores.joblib
 RobertaClassifier.joblib
 svmClassifier.joblib
 test_dataset.csv
 test_loader.joblib
 test_loader.pkl
 tfidf_1000.joblib
 tfidf_100.joblib
 tfidf_10.joblib
 tfidf_1200.joblib
 tfidf_1500.joblib
 tfidf_2000.joblib
 tfidf_2500.joblib
 tfidf_3000.joblib
 tfidf_500.joblib
 tfidf_50.joblib
 train_dataset.csv
 tra

In [ ]:
from vba_pipeline import *
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from utils import save_loader

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, recall_score
from sklearn.model_selection import GridSearchCV
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from joblib import load

from vba_pipeline import create_pipeline, evaluate_model

In [ ]:
param_grid_adaboost = {
    'model__n_estimators': [50, 100, 150],
    'model__learning_rate': [0.01, 0.1, 1.0, 1.5],
    'model__algorithm': ['SAMME', 'SAMME.R']
}

mapper = {
    'white' : 1,
    'mal' : 0
}

In [ ]:
train_set = pd.read_csv('train_dataset.csv', encoding='utf-16le')
val_set = pd.read_csv('validation_dataset.csv', encoding='utf-16le')
test_set = pd.read_csv('test_dataset.csv', encoding='utf-16le')

In [ ]:
x_train, y_train = train_set['vba_code'], train_set['label']
x_val, y_val = val_set['vba_code'], val_set['label']
x_test, y_test = test_set['vba_code'], test_set['label']

y_train = y_train.map(mapper)
y_val = y_val.map(mapper)
y_test = y_test.map(mapper)

In [ ]:
adaBoost_pipeline = create_pipeline(AdaBoostClassifier(), TfidfVectorizer())

In [ ]:
scorer = make_scorer(recall_score, average='binary')

grid_search_adaBoost = GridSearchCV(estimator=adaBoost_pipeline, param_grid=param_grid_adaboost, scoring=scorer, cv=5)

grid_search_adaBoost

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        <vba_pipeline.PreprocessTransformer object at 0x780dacb8f340>),
                                       ('vectorizer', TfidfVectorizer()),
                                       ('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('model', AdaBoostClassifier())]),
             param_grid={'model__algorithm': ['SAMME', 'SAMME.R'],
                         'model__learning_rate': [0.01, 0.1, 1.0, 1.5],
                         'model__n_estimators': [50, 100, 150]},
             scoring=make_scorer(recall_score, response_method='predict', average=binary))

In [ ]:
grid_search_adaBoost.fit(x_train, y_train)

print("Best parameters found: ", grid_search_adaBoost.best_params_)
print("Best recall score: ", grid_search_adaBoost.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1

Best parameters found:  {'model__algorithm': 'SAMME', 'model__learning_rate': 0.01, 'model__n_estimators': 150}
Best recall score:  0.9995023908451521


In [ ]:
best_params = grid_search_adaBoost.best_params_
adaboost_params = {k.replace('model__', ''): v for k, v in best_params.items()}
adaboost_model = AdaBoostClassifier(**adaboost_params)

x_train_tfidf_1000 = load('x_train_1000.joblib')

adaboost_model.fit(x_train_tfidf_1000, y_train)

AdaBoostClassifier(algorithm='SAMME', learning_rate=0.01, n_estimators=150)

In [ ]:
save_loader('/content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection/AdaBoostClassifier.joblib' ,adaboost_model)

AdaBoostClassifier(algorithm='SAMME', learning_rate=0.01, n_estimators=150) saved sucessfuly
